# Back Of Words --> Transferlearning
by Niels Helsø 
git = https://github.com/slein89/BOW_transferlearning  
linkin = https://www.linkedin.com/in/nielshelsoe/

# Agenda
1. Introduktion
2. BoW (Back Of Words)
3. Word embedings
4. Transferlearning

## Code, articels, turtorials and stuf
Git url = https://github.com/slein89/BOW_transferlearning

# Introduktion
- Who am i?
    - Name = Niels
    - Age = 29
    - Education = cand. it
    - Favorite guilty plasure = paagen gifler
    - have code in python for 2 years time
    - Love NLP


# The Danish Business Authority
- 550 people
- Goal: The best conditions and framework for Danish companies
- Data-Scinece team of 5
- Profils:
    - fine arts
    - economis
    - engineer
    - phd
    - it
    - informations studies

# why this talk?
1. Give a overview of where NLP is going and where it have been
2. Explain what the pro and cons are of three diferrent methods
3. I hope to demystify NLP

# PLEASE ASK QUESTIONS 
There are not such a thing as dum questions.

# BoW (Back Of Words)
- one of the first NLP approach
- convert tekst into a vector

In [1]:
corpus=['Pydatacopenhagen is the best place to be tonight, yes tonight',
'At Pydatacopenhagen every one is welcome']

In [2]:
from sklearn.feature_extraction.text import CountVectorizer
vectorizer = CountVectorizer()
#the vector representation of the words (13)
print( vectorizer.fit_transform(corpus).todense() )
#count of the uniq words in corpus
print( vectorizer.vocabulary_ )

[[0 1 1 0 1 0 1 1 1 1 2 0 1]
 [1 0 0 1 1 1 0 1 0 0 0 1 0]]
{'pydatacopenhagen': 7, 'is': 4, 'the': 8, 'best': 2, 'place': 6, 'to': 9, 'be': 1, 'tonight': 10, 'yes': 12, 'at': 0, 'every': 3, 'one': 5, 'welcome': 11}


## ngrams
- helps with preserve some meaning for the sentence

In [3]:
corpus=['Pydatacopenhagen is the best place to be tonight, yes tonight',
'At Pydatacopenhagen every one is welcome']
from nltk import ngrams
grams = ngrams(corpus[0].split(),2)
for gram in grams:
    print (gram)

('Pydatacopenhagen', 'is')
('is', 'the')
('the', 'best')
('best', 'place')
('place', 'to')
('to', 'be')
('be', 'tonight,')
('tonight,', 'yes')
('yes', 'tonight')


# Data (thansk to Prayson Wilfred Daniel)
- trustpilot data
- 254464 (after making a equal distrubiton)
- task: build a sentiment classefier 

In [ ]:
# %load src/preprocessing/text_pre.py
import re
import string
def clean_text (df, row_name):
    #lower all text
    df[row_name] = df[row_name].str.lower()
    #remove all numbers
    df[row_name] = df[row_name].apply(lambda x: x.translate(str.maketrans('','','0123456789')))
    #remove all special chareters
    df[row_name] = df[row_name].apply(lambda x: re.sub('[\W]+', ' ', x))
    #make translation tabel where punctuation is removed and apply it to the text
    table = str.maketrans({key: None for key in string.punctuation})
    df[row_name] = df[row_name].apply(lambda x: x.translate(table))
    return df




In [4]:
from src.data.load_data import load_trustpilot_data
from src.preprocessing.text_pre import clean_text
df_trust = load_trustpilot_data()
df_trust = clean_text(df_trust, 'text')
print('Amount of the to classes')
print('-----------------')
print(df_trust['y'].value_counts())
print('')
print('Examples of text')
print('-----------------')
for index,row in df_trust[:5].iterrows():
    print(row['text'])

Amount of the to classes
-----------------
1    127232
0    127232
Name: y, dtype: int64

Examples of text
-----------------
det var iorden 
nåede sikkert og hurtigt frem det var en god oplevelse
vi har altid været glade for tryg og deres måde at håndtere forsikringssager på vi anmelder self heller ikke hvad som helst men til gengæld får vi altid en ærlig og redelig behandling uden problemer selv om tryg i perioder kan være lidt dyrere end andre forsikringsselskaber foretrækker vi dem fordi vi så heller ikke skal trækkes med diskussioner om udbetaling når uheldet er ude 
hurtig pakke lev til pakkeboksen
bestilte tøj på nettet og fik pakken hurtigere end ventet og kunne endda følge den hele vejen 


In [5]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(df_trust['text'], df_trust['y'], random_state=42 )

In [6]:
from src.preprocessing.metrics import get_metrics

Metrics:
- Roc auc
- Confusion Matrix
- Classification report (recall, precession, F1 score)

### performanche on BoW
lets se how BOW performs

In [7]:
from joblib import load
model_bow = load('data/models/bow_model.pkl')
get_metrics(model_bow, X_test, y_test)

roc_auc_score
0.9001972503652068
-------------------
classification_report
              precision    recall  f1-score   support

           0       0.90      0.90      0.90     31932
           1       0.90      0.90      0.90     31684

   micro avg       0.90      0.90      0.90     63616
   macro avg       0.90      0.90      0.90     63616
weighted avg       0.90      0.90      0.90     63616

-------------------
confusion_matrix
[[28623  3309]
 [ 3041 28643]]
-------------------
CPU times: user 14.6 s, sys: 237 ms, total: 14.8 s
Wall time: 14.8 s


## Some issues
- Ngrams get us some of the way
- Stil just one vector per sentens (document)

# Word embdings
Word2Vec has two methods:
1. CBOW
2. Skip-gram

In [2]:
from IPython.display import IFrame
IFrame("pictures/skipgra, and cbow.png", width=600, height=400)

Lets train one and have a look of how good it is at finding word similarity

In [9]:
import gensim
word2vec_model = gensim.models.Word2Vec.load("data/models/word2vec.model")
w1 = 'nokia'
word2vec_model.wv.most_similar(positive=w1)

[('lumia', 0.7854803204536438),
 ('htc', 0.7818347811698914),
 ('iphone', 0.7195836305618286),
 ('sony', 0.6978476643562317),
 ('lenovo', 0.692030668258667),
 ('huawei', 0.6810896396636963),
 ('bærbar', 0.6735221147537231),
 ('tablet', 0.6730877161026001),
 ('mobiltelefon', 0.6699701547622681),
 ('ipod', 0.6555711627006531)]

## FastText
- Framework with word2vecs cbow implementet 
  - ngrams (both on words and characters)
  - size (size of the context window)
  - dim (size of the vector)
- Each word is represented as a bag of character n-grams
    - ngrams = 3
    - where = <wh, whe, her, ere, re>

In [11]:
%%time
from src.model.fasttext import fasttext_pipeline 
X_train,X_test,y_train,y_test = train_test_split(df_trust[['text']], df_trust['y'], random_state=42 )
model_fasttext = fasttext_pipeline(X_train, y_train)
get_metrics(model_fasttext, X_test, y_test)

roc_auc_score
0.9457883754461629
-------------------
classification_report
              precision    recall  f1-score   support

           0       0.94      0.96      0.95     31932
           1       0.96      0.93      0.95     31684

   micro avg       0.95      0.95      0.95     63616
   macro avg       0.95      0.95      0.95     63616
weighted avg       0.95      0.95      0.95     63616

-------------------
confusion_matrix
[[30552  1380]
 [ 2066 29618]]
-------------------
CPU times: user 49min 20s, sys: 9.16 s, total: 49min 29s
Wall time: 4min 22s


# Transferlearning
- lets take somthing that we have learn somwhere else and apply it to a new domain
- why this?
    - we do not always have a ton of data
    - The model do not need to learn from sratch
    - Thus we save time
    - We make use of the models there are bulid by very talented people
    - the apporach have proven it self in computer vission

## Universal Language Model Fine-tuning for Text Classification (ULMFiT)
- fast.ai
- 15 may 2018
- Make use of Language model apporach
  - A model which predict the next word in a sentence
      - e.g. "jeg elsker pågen" -> "gifler" 
  - have high understading of semantic
      - eg. " så det der kommer ..." != "jeg så en flot sol opgang"
  - have a high understading of grammatic
- Data = wikipedia

In [5]:
from IPython.display import IFrame
IFrame("pictures/ulmfit.png", width=600, height=200)

## Status on a danish ulmfit model
- traind for 24 hours on danish wikipidia
- 20000 + wikipidia articels 
- 60000 + tokens
- has for the now a ROC on 80% (train=2400/test=1600)

Lets have a look

# Recap
- BoW
  - pros
      - Robust towards its specific task
      - High transparity
      - Quick to code
  - cons
      - "only" counts word
      - loses semantics
      - slow to train
- Word2vec(FastText)
    - pros
        - take semantics into acount
        - is fast
    - cons
        - transparity is a bit of a blur
        - needs a good purtion of data
- Transferlearning (ULMFIT)
    - pros
        - state of the art performanche (the say)
        - have the potintial to save a lot of time
    - cons
        - hard to code
        - initial training take a lot of time
        - Black Box

# KISS
***Keep It Simpel and Stupid***

Questions?

Thanks for your time.